In [ ]:
###yolov5 install
import os
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5
%cd {HOME}/yolov5
!pip install -r requirements.txt
from IPython import display
display.clear_output()

In [ ]:
##YOLOv8 install
!pip install ultralytics
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
import numpy as np
model = YOLO('yolov8s.pt')

In [ ]:
## Video_Info
import supervision as sv
vid = 'PATH OF VIDEO'
sv.VideoInfo.from_video_path(vid)

In [ ]:
import numpy as np
# initiate polygon zone
## Use this website to mark pixels ---> https://pixspy.com

polygon = np.array([
  [368,367],
  [395,349],
  [539,334],
  [571,357],
  [848,477],
  [544,475],
  [369,382]
])

video_info = sv.VideoInfo.from_video_path(vid)
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# initiate annotators
box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.3, text_padding=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.blue(), thickness=3, text_thickness=2,
                                         text_scale=1, text_padding=2, text_color=sv.Color.white())

def process_frame(frame: np.ndarray, _) -> np.ndarray:
    # detect
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_yolov8(results)
    detections = detections[detections.class_id == 0]
    zone.trigger(detections=detections)

    # annotate
    box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.3, text_padding=2)
    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(scene=frame)

    return frame

sv.process_video(source_path=vid, target_path=f"{HOME}/Ireland.mp4", callback=process_frame)

from IPython import display
display.clear_output()